# Visualize comparison between nno and nob mean scores

In [ ]:
import pandas as pd
from pathlib import Path

from notebook_utils import load_files_to_df, expand_abbreviations

output_dir = Path("../data/output/2024")


df = load_files_to_df(output_dir)

df = expand_abbreviations(df)

columns_to_keep = [
    "cer",
    "wer",
    "sbert_semdist",
    "semdist",
    "aligned_semdist",
    "date",
    "model_name",
    "language_code",
    "prediction_langcode",
    "year",
    "dialect",
    "gender",
]

df = df[columns_to_keep]

data_dict = {
    "modell": [],
    "språk": [],
    "CER": [],
    "WER": [],
    "aligned semantic distance": [],
    "semantic distance": [],
    "semantic distance (sBERT)": [],
}

for (model, lang, pred_lang), df_ in df.groupby(
    ["model_name", "language_code", "prediction_langcode"]
):
    if pred_lang == "":
        continue
    data_dict["modell"].append(model)
    data_dict["språk"].append(lang)
    data_dict["CER"].append(df_.cer.mean())
    data_dict["WER"].append(df_.wer.mean())
    data_dict["aligned semantic distance"].append(df_.aligned_semdist.mean())
    data_dict["semantic distance"].append(df_.semdist.mean())
    data_dict["semantic distance (sBERT)"].append(df_.sbert_semdist.mean())


mean_score_df = pd.DataFrame(data_dict)

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.colors import qualitative

import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(
    [
        html.H1("Visualisering av gjennomsnittscore"),
        dcc.Dropdown(
            id="score-dropdown",
            options=[
                {"label": "CER", "value": "CER"},
                {"label": "WER", "value": "WER"},
                {
                    "label": "aligned semantic distance",
                    "value": "aligned semantic distance",
                },
                {"label": "semantic distance", "value": "semantic distance"},
                {
                    "label": "semantic distance (sBERT)",
                    "value": "semantic distance (sBERT)",
                },
            ],
            value="CER",
        ),
        dcc.Graph(id="bar-chart"),
    ]
)


# Define the callback to update the bar chart
@app.callback(Output("bar-chart", "figure"), Input("score-dropdown", "value"))
def update_bar_chart(selected_score):
    fig = px.bar(
        mean_score_df.sort_values(by=selected_score, ascending=True),
        x="modell",
        y=selected_score,
        color="språk",
        color_discrete_map={
            "nno": qualitative.Dark2[0],
            "nob": qualitative.Dark2[5],
        },
        barmode="group",
        labels={
            "modell": "Modell",
            selected_score: "Gjennomsnittscore",
            "språk": "Språk",
        },
        title=f"Gjennomsnittlig {selected_score}",
    )
    return fig


# Run the app
if __name__ == "__main__":
    app.run(debug=True, jupyter=True, port=8051)